In [1]:
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
import keras
import tensorflow as tf
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout


Using plaidml.keras.backend backend.


In [5]:
train_data_directory = 'D:/Dropbox/Machine Learning/Projects/dogs-vs-cats/data'

In [6]:
# image data preprocessing
image_size = (112,112)

batch_size = 32


class_mode = 'binary' # for multi-class classification problem, use: class_mode = 'category' 

from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    fill_mode="nearest",
    validation_split=0.2 # set validation split
    ) 

train_generator = train_datagen.flow_from_directory(
    train_data_directory,
    target_size=image_size,
    batch_size=batch_size,
    class_mode=class_mode, # for multi-class classification problem, use 'category'
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_directory, # same directory as training data
    target_size=image_size,
    batch_size=batch_size,
    class_mode=class_mode, # for multi-class classification problem, use 'category'
    subset='validation') # set as validation data

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [11]:
# build a model
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten

VGG_16 = Sequential()
VGG_16.add(Conv2D(input_shape=(112,112,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
VGG_16.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
VGG_16.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

VGG_16.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
VGG_16.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
VGG_16.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

VGG_16.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
VGG_16.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
VGG_16.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
VGG_16.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

VGG_16.add(Flatten())

VGG_16.add(Dense(1024,activation="relu"))
VGG_16.add(Dense(1, activation="sigmoid"))

VGG_16.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 112, 112, 64)      1792      
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 112, 112, 64)      36928     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 56, 56, 128)       73856     
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 56, 56, 128)       147584    
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 28, 28, 128)       0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 28, 28, 256)       295168    
__________

In [12]:
# compile model
from keras import optimizers

VGG_16.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [13]:
# Callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("vgg16_1.h5", 
                             monitor='val_acc', 
                             verbose=1, 
                             save_best_only=True, 
                             save_weights_only=False, 
                             mode='auto', 
                             period=1)
early = EarlyStopping(monitor='val_acc', 
                      min_delta=0, 
                      patience=5, 
                      verbose=1, 
                      mode='auto')

In [17]:
# fit/train model

nb_epochs = 100

history = VGG_16.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs)

Epoch 1/100


INFO:plaidml:Analyzing Ops: 164 of 331 operations complete


625/625 [==============================] - 1112s 2s/step - loss: 0.6239 - acc: 0.6483 - val_loss: 0.5734 - val_acc: 0.6955
Epoch 2/100
625/625 [==============================] - 1078s 2s/step - loss: 0.5442 - acc: 0.7210 - val_loss: 0.4913 - val_acc: 0.7685
Epoch 3/100
625/625 [==============================] - 1057s 2s/step - loss: 0.4827 - acc: 0.7679 - val_loss: 0.4717 - val_acc: 0.7687
Epoch 4/100
625/625 [==============================] - 1088s 2s/step - loss: 0.4332 - acc: 0.8012 - val_loss: 0.4182 - val_acc: 0.8134
Epoch 5/100
625/625 [==============================] - 1115s 2s/step - loss: 0.3904 - acc: 0.8242 - val_loss: 0.3572 - val_acc: 0.8426
Epoch 6/100
625/625 [==============================] - 1135s 2s/step - loss: 0.3517 - acc: 0.8450 - val_loss: 0.3275 - val_acc: 0.8583
Epoch 7/100
625/625 [==============================] - 1096s 2s/step - loss: 0.3222 - acc: 0.8602 - val_loss: 0.3139 - val_acc: 0.8738
Epoch 8/100
625/625 [==============================] - 1023s 2s/ste

KeyboardInterrupt: 